In [5]:
import tweepy
import json
import psycopg2
import cred
from geopy.geocoders import Nominatim

In [6]:
consumer_key = cred.api_key
consumer_secret = cred.api_secrect_key
access_token = cred.access_token
access_token_secret = cred.access_token_secrest

In [7]:
def api_connection():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
        print("auth ok")
    except Exception as e:
        print(e.Exception)

    return api

In [8]:
api = api_connection()

auth ok


In [9]:
def extract_tweets(hashtag, count, geo):
    #api = api_connection()
    print('Inicia..')
    tweets_json = []
    tuits_cursor = tweepy.Cursor(api.search,q=hashtag, lang='es', result_type='recent', rpp=100, include_rts=False,tweet_mode='extended',geocode=geo).items(count)
    #a = tuits_cursor.next()
    for tweet in tuits_cursor:
        if (not tweet.retweeted and 'RT @' not in tweet.full_text):
            tweets_json.append(tweet._json)
            #avatar, geo_loc, verificada?, tag, username, user_id, entities(img, url, etc), source(android,ios,web)
    return tweets_json

In [10]:
def apply_filter(topic):
    topic = topic+since_until
    topic = topic+filter_retweets
    return topic

In [11]:
def connect_db(user,password,host,port,database):
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")              
    return cursor,connection

In [38]:
cursor,connection = connect_db(cred.database_user,cred.database_pass,"192.168.0.22",'5432','data_tweets')

You are connected to -  ('PostgreSQL 12.4 (Ubuntu 12.4-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-10ubuntu2) 9.3.0, 64-bit',) 



In [33]:
#geo = '-34.6025287,-58.3796486' # buenos aires, argentina
#geo = '-23.5861315,-46.6582803' # sao paulo, brasil
#geo = '-34.8207362,-56.3765221' #montevideo, uruguay
#geo = '-32.8832979,-68.8760287' # mendoza, argentina
#geo = '-31.3992876,-64.2643839' #cordoba, argentina
#geo = '-31.6145747,-60.6980372' # santa fe, argentina
#geo = '7.7721289,-72.226306' # San cristobal , venezuela
#geo = '10.2241386,-67.3411383' # La victoria, Aragua , venezuela
geo = '10.4686988,-67.0304528' #caracas, venezuela
geo_t = geo+',400km'
topico_buscar_a ='trump'
topico_buscar_b ='biden'
since_until=' since:2020-10-03 until:2020-10-07'
count = 5000
filter_retweets=" -filter:retweets"

In [34]:
geolocator = Nominatim(user_agent="twitter_scrapper")
location = geolocator.reverse(geo)
location.raw['address']['city'],location.raw['address']['country']

('Caracas', 'Venezuela')

In [35]:
tweets_a= extract_tweets(apply_filter(topico_buscar_a),count, geo_t)

Inicia..
Rate limit reached. Sleeping for: 1071


In [36]:
tweets_b= extract_tweets(apply_filter(topico_buscar_b),count, geo_t)

Inicia..
Rate limit reached. Sleeping for: 1074


In [37]:
print(len(tweets_a),len(tweets_b))

4999 2469


In [30]:
def insert_tweets(tweet, topic,cursor,connection):
    #json.dumps(x)
    script = "INSERT INTO data_tweets (ID, tweet , topic, city_searched, country_searched) VALUES (%s, %s, %s, %s, %s) ON CONFLICT (ID) DO NOTHING;;"
    cursor.execute(script,(tweet['id'],json.dumps(tweet),topic, location.raw['address']['city'], location.raw['address']['country'] ))
    connection.commit()


In [31]:
for tweet in tweets_a:
    insert_tweets(tweet,topico_buscar_a,cursor,connection)

In [32]:
for tweet in tweets_b:
    insert_tweets(tweet,topico_buscar_b,cursor,connection)